In [ ]:
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from scipy.stats import pearsonr
from IPython.display import display
from urllib.parse import urlparse, parse_qs, urlunparse
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Get user input for the URL
user_input_url = input("Enter the tournament URL: ")
# Append the "&zeilen=99999" string to the user-provided URL, this expands the final rankings to include all players beyond row 150
url_to_scrape = user_input_url + "&zeilen=99999"

# Record start time
start_time = time.time()

# Parse the URL
url_components = urlparse(user_input_url)
query_params = parse_qs(url_components.query)

# Modify the 'art' parameter to 0
query_params['art'] = ['0']

# Remove the 'rd' parameter
if 'rd' in query_params:
    query_params.pop('rd')

# Create the modified URL from which to extract rating data (FIDE IDs are not available on the standings page in chess-results!)
modified_query = '&'.join(
    [f"{key}={value[0]}" for key, value in query_params.items()])
modified_url = urlunparse(
    (url_components.scheme, url_components.netloc, url_components.path,
     url_components.params, modified_query, url_components.fragment))

print('Fetching FIDE ID data from', modified_url + "&zeilen=99999")

# Send a GET request and parse the HTML content
response = requests.get(url_to_scrape)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all h2 tags
h2_tags = soup.find_all('h2')

# Find the tournament name from the first <h2> tag
tournament_name = soup.find('h2').text.strip()


def scrape_fide_ids(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the first table after the second h2 tag, this is where our standings are housed
    table = soup.find_all('h2')[1].find_next('table')

    # Find the first row of the table
    first_row = table.find('tr')

    # Find the index of the "FideID" in the first row
    cells = first_row.find_all('td')
    fide_id_index = None
    for index, cell in enumerate(cells):
        if "FideID" in cell.text:
            fide_id_index = index
            break

    # Extract FIDE IDs from the "FideID" column
    fide_ids = []
    rows = table.find_all('tr')[1:]  # Exclude the first row (header)
    for row in rows:
        cells = row.find_all('td')
        if fide_id_index < len(cells):
            fide_id = cells[fide_id_index].text.strip()
            fide_ids.append(fide_id)

    return fide_ids


# Building the full URL that shows the entire starting list and extracting all FIDE IDs of the participants, in descending order of Elo
url = modified_url + "&zeilen=99999"
fide_ids = scrape_fide_ids(url)

# Check if there are at least two h2 tags
if len(h2_tags) >= 2:
    # Get the second h2 tag
    second_h2 = h2_tags[1]

    # Find the table following the second h2 tag
    table = second_h2.find_next('table')

    # Find all rows in the table
    rows = table.find_all('tr')

    # Initialize empty lists to store data
    rk_data = []
    sno_data = []
    rtg_data = []

    # Identify the positions of 'Rk.', 'SNo', and 'Rtg' in the header
    header_row = rows[0]  # Header row is the first row
    header_cells = header_row.find_all(['th', 'td'])

    rk_index = None
    sno_index = None
    rtg_i_index = None
    rtg_index = None

    for index, cell in enumerate(header_cells):
        if 'Rk.' in cell.get_text():
            rk_index = index
        elif 'SNo' in cell.get_text():
            sno_index = index
        elif 'RtgI' in cell.get_text():
            rtg_i_index = index
        elif 'Rtg' in cell.get_text():
            rtg_index = index

# Check if 'Rk.', 'SNo', 'RtgI', and 'Rtg' were found in the header
if rk_index is not None and sno_index is not None and (
        rtg_i_index is not None or rtg_index is not None):
    # Loop through rows and extract data
    for row in rows[1:]:  # Skip header row
        columns = row.find_all(['td', 'th'])
        rk_data.append(
            int(''.join(columns[rk_index].text.split())) if columns[rk_index].
            text.strip() else 0)
        sno_data.append(int(columns[sno_index].text.strip()))
        if rtg_i_index is not None:
            rtg_data.append(int(columns[rtg_i_index].text.strip()))
        elif rtg_index is not None:
            rtg_data.append(int(columns[rtg_index].text.strip()))

# Create a DataFrame from the collected data
data = {'Rk.': rk_data, 'SNo': sno_data, 'Rtg': rtg_data}
df = pd.DataFrame(data)

# Sort the DataFrame by the 'SNo' column - IMPORTANT STEP, DO NOT REMOVE.
df.sort_values(
    by='SNo', inplace=True
)  # If you append FIDE IDs without sorting, you'll scramble the data from the next step!

# Append FIDE IDs to the DataFrame sorted by SNO
df['FideID'] = fide_ids

# Filter out rows where 'Rk.' is 0, as those are early withdrawals and tournament no-shows
df = df[df['Rk.'] != 0]

# Load the pre-processed urs_ratings_df DataFrame
urs_ratings_df = pd.read_pickle('urs_ratings.pkl')

# Make sure both DataFrames have the FideID entries as strings
df['FideID'] = df['FideID'].astype(str)
urs_ratings_df['FideID'] = urs_ratings_df['FideID'].astype(str)

# Merge the 'URating' column by matching on 'FideID' as the index
df = df.merge(urs_ratings_df, on='FideID', how='left')

# Sort the DataFrame by 'URating' in descending order
df.sort_values(by='URating', ascending=False, inplace=True)

# Add a new column 'Rank' with player ranks based on URS ratings
df['URank'] = range(1, len(df) + 1)

# Calculate correlation coefficients
Elo_correlation = abs(pearsonr(df['SNo'], df['Rk.'])[0])**2
URS_correlation = abs(pearsonr(df['URank'], df['Rk.'])[0])**2


# Function to calculate positive and negative outliers
def count_outliers(data1, data2, threshold):
    positive_outliers = sum((data2 - data1) / len(data1) > threshold)
    negative_outliers = sum((data2 - data1) / len(data1) < -threshold)
    return positive_outliers, negative_outliers


# Calculate positive and negative outliers thresholds
threshold = 0.33  # An outlier is one that "Crossed over" thru at least 1/3 of the tournament list.

# Calculate summary stats based on a threshold (hardcoded above, can be modified if you want to).
num_players = len(df)
positive_outliers_elo, negative_outliers_elo = count_outliers(
    df['Rk.'], df['SNo'], threshold)
positive_outliers_urs, negative_outliers_urs = count_outliers(
    df['Rk.'], df['URank'], threshold)

# Print summary stats in boldface and some color formatting for readability
print(
    f"\033[1mFinal rankings R² correlation of {tournament_name} - by Elo: {Elo_correlation:.4f}, by URS: {URS_correlation:.4f} \nSummary stats: N={num_players}, Φ-Elo = {df['Rtg'].mean():.0f}, Φ-URS = {df['URating'].mean():.0f}\nOutliers: \033[1m\033[32m{positive_outliers_elo} Elo positive, {positive_outliers_urs} URS positive\033[0m, \033[1m\033[31m{negative_outliers_elo} Elo negative, {negative_outliers_urs} URS negative\033[0m"
)

# Calculate the outlier range for Elo
elo_outlier_range = threshold * len(df)


# Define a function to apply highlighting and formatting based on relation
def format_row(row):
    elo_diff = row['SNo'] - row['Rk.']

    if elo_diff >= elo_outlier_range:
        return ['color: green; font-weight: bold'] * len(row)
    elif elo_diff <= -elo_outlier_range:
        return ['color: red; font-weight: bold'] * len(row)
    else:
        return [''] * len(row)


# Sort the DataFrame in place by 'Rk.' column
df.sort_values(by='Rk.', inplace=True)

# Filter the DataFrame to get only the outliers
outliers_df = df[df.apply(lambda row: format_row(row)[0] != '', axis=1)]

# Count the green and red rows
green_count = len(outliers_df[outliers_df.apply(
    lambda row: format_row(row)[0] == 'color: green; font-weight: bold',
    axis=1)])
red_count = len(outliers_df[outliers_df.apply(
    lambda row: format_row(row)[0] == 'color: red; font-weight: bold',
    axis=1)])

# Apply the format function to each row
styled_outliers_df = outliers_df.style.hide(axis="index").apply(format_row, axis=1)\
    .set_table_styles([
        {'selector': 'thead', 'props': [('background-color', '#e6e6fa')]},
        {'selector': 'th, td', 'props': [('text-align', 'center')]},  # Center text in cells
        {'selector': 'th, td', 'props': [('border', '1px solid black')]}  # Add gridlines
    ])\
    .format({'URating': '{:.0f}'})\
    .set_properties(**{'text-align': 'center', 'padding': '8px'})

# Display the styled outliers DataFrame
display(styled_outliers_df)

# Create a scatter plot of Rk. vs SNo.
plt.figure(figsize=(9, 5))

max_annotations = 5  # Maximum number of annotations

if positive_outliers_elo <= max_annotations:  # Set limit under which the names of the positive outliers will be annotated on the plot
    draw_annotations = True
else:
    draw_annotations = False  # To prevent overcrowding of the plot area

for index, row in df.iterrows():
    elo_diff = row['SNo'] - row['Rk.']

    if elo_diff >= elo_outlier_range:
        plt.scatter(row['SNo'],
                    row['Rk.'],
                    marker='*',
                    color='green',
                    alpha=0.5)

        if draw_annotations == True:
            plt.annotate(
                row['PlayerName'],  # Display PlayerName as annotation
                xy=(row['SNo'], row['Rk.']),  # Position of the point
                xytext=(0, -10),  # Text position relative to the point
                textcoords='offset points',
                ha='center',  # Horizontally center the text
                fontsize=8,
                color='green')

    elif elo_diff <= -elo_outlier_range:
        plt.scatter(row['SNo'], row['Rk.'], marker='x', color='red', alpha=0.5)

    else:
        plt.scatter(row['SNo'],
                    row['Rk.'],
                    marker='.',
                    color='blue',
                    alpha=0.3)

# Fit a linear regression model
X = df['SNo'].values.reshape(-1, 1)
y = df['Rk.'].values
model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)  # Predict the linear fit line

plt.plot(X, y_pred, color='red', linewidth=2,
         label='Linear Fit')  # Plot the linear fit line

r2 = r2_score(y, y_pred)  # Calculate R-squared value

# Annotate the R-squared value on the plot
plt.annotate(f'$R^2$ = {r2:.4f}',
             xy=(0.05, 0.85),
             xycoords='axes fraction',
             fontsize=12)

legend_elements = [
    Line2D([0], [0], color='red', lw=2, label='Linear Fit'),
    Line2D([], [],
           marker='.',
           color='blue',
           label='Regular Player',
           linestyle='None',
           alpha=0.3,
           markersize=8),
    Line2D([], [],
           marker='*',
           color='green',
           label='Positive Outlier',
           linestyle='None',
           alpha=0.5,
           markersize=6),
    Line2D([], [],
           marker='x',
           color='red',
           label='Negative Outlier',
           linestyle='None',
           alpha=0.5,
           markersize=6),
]

# Set plot labels and title
plt.xlabel('SNo')
plt.ylabel('Rk.')
plt.title(f'Scatter Plot of {tournament_name} with Linear Fit')
plt.legend(handles=legend_elements, loc='lower right')
plt.show()

# Calculate and print runtime evaluation
end_time = time.time()
runtime = end_time - start_time
print(f"Script runtime: {runtime:.2f} seconds")